In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from GAD_util import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"running on {device}")

running on cuda


In [ ]:
sequence_length = 30
dt = 1/250
learning_rate = 0.005
batch_size = 10000
batch_num= 20
epoch_num = 300
T = dt * sequence_length

In [ ]:
loss_fn = loss_exp_OCE(1.3,X_max=True, p0_mode='calculate')

In [ ]:
# Load the network
network = RNN_BN_simple(sequence_length=sequence_length).to(device=device)
network.eval()
# network.load_state_dict(torch.load('../Result/{name}'))

In [ ]:
# Evaluation on the tested price trajectory
def PnL(price_single):
    # Calculate the PnL for a single price trajectory
    holding = network(price_single[:-1].unsqueeze(-1).unsqueeze(0)).squeeze()
    delta_price = price_single[1:] - price_single[:-1]
    Pnl = (holding * delta_price.squeeze()).sum().item()
    payoff = max((price_single.mean()-10).item(),0)
    return (Pnl-payoff)
data = pd.read_csv("../Data/stocks_close_prices_2008_2021.csv")
company = 'MSFT'
test_period= data[company].values[2880:2911]
test_period = test_period / test_period[0] * 10
price_single_test = torch.tensor(test_period, dtype=torch.float32, device=device)

result = PnL(price_single_test)
print(result)

In [ ]:
# Evaluation on the dataset
def performance_dataset(price):
    # Compute the loss for the given price data
    holding = network(price[:,:-1].unsqueeze(-1)).squeeze()
    loss = loss_fn(holding, price)
    return loss.item()
# load data
company = 'MSFT'
run_name = f'GAD_{company}'
price = torch.load(f"../Data/{run_name}/GAD_fix_val.pt").to(device)
# price = torch.load(f"../Data/{run_name}/GAD_fix_test.pt").to(device)
# price = torch.load(f"../Data/{run_name}/GAD_fix_train.pt").to(device)
# price = torch.load(f"../Data/{run_name}/GAD_robust_val.pt").to(device)
# price = torch.load(f"../Data/{run_name}/GAD_robust_test.pt").to(device)
# price = torch.load(f"../Data/{run_name}/GAD_robust_train.pt").to(device)
# price = torch.load(f"../Data/{run_name}/GAD_real_test.pt").to(device)

# Evaluate the performance of the network
result = performance_dataset(price)
print(result)